In [ ]:
import streamlit as st
import pandas as pd
import numpy as np
from faker import Faker
import random
import json

from snowflake.cortex import complete
from snowflake.snowpark.context import get_active_session

session = get_active_session()

In [ ]:
#Synthetically create a dataframe of customers, products, contract val, and received revenue

def generate_customer_data():
    """Generates mock customer data."""

    data = []
    product_list =["Cortex LLM","Cortex Search",
                   "Cortex Analyst","Cortex Agents",
                "DocAI","Model Registry"]


    customer_list = ["Apple", "Amazon", "Microsoft", "Alphabet", "Google", "Facebook", "Procter & Gamble", "Coca-Cola", 
                         "ExxonMobil", "Verizon", "Johnson & Johnson", "Pfizer", "UnitedHealth Group", "Caterpillar", 
                         "3M", "Chevron", "Walmart", "McDonald's", "AT&T", "Berkshire Hathaway", "Sears Holdings", "Cargill", "DuPont", 
                         "General Electric", "Merck", "Visa", "Intel", "Cisco Systems", "Wealthsimple", "Snowflake"]
        
    for i in customer_list:
        customer_name = i
        products_used = random.sample(product_list
,
            random.randint(1, 4),
        )
        selected_product = random.sample(product_list, 1)[0]
        total_contract_value = round(random.uniform(1000, 100000), 2)
        revenue_received = round(random.uniform(0, total_contract_value), 2)

        data.append(
            {
                "CUSTOMER": customer_name,
                "TOTAL_CONTRACT_VALUE": total_contract_value,
                "REVENUE_RECEIVED": revenue_received,
                "PRODUCTS_USED": products_used,
                "PROPENSITY_MODEL_SUGGESTED_PRODUCT": selected_product,

            }
        )
    df = pd.DataFrame(data)
    return df

# Generate and display the DataFrame
df = generate_customer_data()
df.tail(10)

In [ ]:
# session.write_pandas(df, "CUSTOMER_DATA", auto_create_table=True, quote_identifiers = False, overwrite=False)

In [ ]:
SHOW TABLES;

In [ ]:
CREATE STAGE IF NOT EXISTS SEMANTIC DIRECTORY = ( ENABLE = true );

# We will now create the semantic model to query our new data with natural language

* To do so, first go to the AI Studio in snowsight and click the Cortex Analyst Tab

* Choose the appropriate database and schema the select the SEMANTIC stage we just created

* Click Create New

* Fill out the Description - 
    * "Semantic model containing information about customer product data including the customer name, which products they currently use, their total contract value, and the amount of revenue we have received from the customer"

* Select the CUSTOMER_PRODUCT_DATA table and select all columns


In [ ]:
data = session.table("CUSTOMER_DATA")
data.show(5)

In [ ]:
## Start testing cortex analyst api call here
from typing import List, Dict, Optional
import _snowflake


class CortexAnalyst():
    def __init__(self, db: str, schema: str, stage: str, semantic_model_file_path: str):
        self.db = db
        self.schema = schema
        self.stage = stage
        self.semantic_model_file_path = semantic_model_file_path


    # @instrument
    def send_message(self, prompt: str) -> dict:

        """Calls the REST API and returns the response."""
        
        messages = []
        messages.append({"role": "user", "content": [{"type": "text", "text": prompt}]})
        request_body = {
            "messages": messages, #need to wrap in a list?
            "semantic_model_file": f"@{self.db}.{self.schema}.{self.stage}/{self.semantic_model_file_path}",
        }

        print(request_body)

        resp = _snowflake.send_snow_api_request(
            "POST",
            f"/api/v2/cortex/analyst/message",
            {},
            {},
            request_body,
            {},
            30000,
        )
        if resp["status"] < 400:
            return json.loads(resp["content"])
        else:
            # messages.pop()
            raise Exception(
                f"Failed request with status {resp['status']}: {resp}"
            )
    # @instrument
    def process_message(self, prompt: str) -> None:
        """Processes a message and adds the response to the chat."""
        messages=[]
        messages.append(
            {"role": "user", "content": [{"type": "text", "text": prompt}]}
        )
        with st.chat_message("user"):
            st.markdown(prompt)
        with st.chat_message("assistant"):
            with st.spinner("Generating response..."):
                # response = "who had the most rec yards week 10"
                response = self.send_message(prompt=prompt)
                request_id = response["request_id"]
                content = response["message"]["content"]
                messages.append(
                    {**response['message'], "request_id": request_id}
                )
        return self.display_content(content=content, request_id=request_id, prompt = prompt)  # type: ignore[arg-type]
        # return response
    
    # @instrument
    def display_content(self,
        content: List[Dict[str, str]],
        request_id: Optional[str] = None,
        message_index: Optional[int] = None,
        prompt: Optional[str] = None,
    ) -> None:
        """Displays a content item for a message."""
        message_index = message_index or len(prompt)
        # if request_id:
            # with st.expander("Request ID", expanded=False):
                # st.markdown(request_id)
        for item in content:
            if item["type"] == "text":
                st.markdown(item["text"])
            elif item["type"] == "suggestions":
                with st.expander("Suggestions", expanded=True):
                    for suggestion_index, suggestion in enumerate(item["suggestions"]):
                        if st.button(suggestion, key=f"{message_index}_{suggestion_index}"):
                            st.session_state.active_suggestion = suggestion
            elif item["type"] == "sql":
                return self.display_sql(item["statement"])

    # @instrument
    def display_sql(self, sql: str) -> None:
        with st.expander("SQL Query", expanded=False):
            st.code(sql, language="sql")
        with st.expander("Results", expanded=True):
            with st.spinner("Running SQL..."):
                session = get_active_session()
                df = session.sql(sql).to_pandas()
                if len(df.index) > 1:
                    data_tab, line_tab, bar_tab = st.tabs(
                        ["Data", "Line Chart", "Bar Chart"]
                    )
                    data_tab.dataframe(df)
                    if len(df.columns) > 1:
                        df = df.set_index(df.columns[0])
                    with line_tab:
                        st.line_chart(df)
                    with bar_tab:
                        st.bar_chart(df)
                else:
                    st.dataframe(df)

        return df

CA = CortexAnalyst(db='TR_MULTI_AGENT', schema='PUBLIC', stage='SEMANTIC', semantic_model_file_path='customer_data.yaml')

In [ ]:
CA_response = CA.process_message(prompt='Show me all the products for Wealthsimple')

In [ ]:
CA_response = CA.process_message(prompt='Which product should I pitch to Welathsimple?')

## Define Retrieval Service
### Get up to date product info and customer references on products to inform sales pitches

In [ ]:
LS @DOCUMENTS

In [ ]:
CREATE TABLE TR_MULTI_AGENT.PUBLIC.DOC_TEXT_TABLE IF NOT EXISTS (relative_path VARCHAR(500), raw_text VARIANT);

In [ ]:
INSERT INTO TR_MULTI_AGENT.PUBLIC.DOC_TEXT_TABLE (relative_path, raw_text)
WITH html_files AS (
    SELECT DISTINCT
        METADATA$FILENAME AS relative_path
    FROM @TR_MULTI_AGENT.PUBLIC.DOCUMENTS
    -- WHERE METADATA$FILENAME ILIKE '%.pdf'
    --   -- Exclude files that have already been parsed
      WHERE METADATA$FILENAME NOT IN (SELECT relative_path FROM DOC_TEXT_TABLE)
)
SELECT 
    relative_path,
    SNOWFLAKE.CORTEX.PARSE_DOCUMENT(
        '@TR_MULTI_AGENT.PUBLIC.DOCUMENTS',  -- Your stage name
        relative_path,  -- File path
        {'mode': 'layout'}  -- Adjust mode as needed ('layout', 'ocr')
    ) AS raw_text
FROM html_files;

In [ ]:
CREATE TABLE IF NOT EXISTS TR_MULTI_AGENT.PUBLIC.DOC_CHUNKS_TABLE AS
WITH text_chunks AS (
    SELECT
        relative_path,
        SNOWFLAKE.CORTEX.SPLIT_TEXT_RECURSIVE_CHARACTER(
            raw_text:content::STRING,  -- Extract the 'content' field from the JSON
            'markdown',    -- Adjust to 'markdown' if needed
            2000,       -- Adjust chunk size
            100,       -- Adjust overlap size
            ['\n\n']     -- Adjust separators
        ) AS chunks
    FROM TR_MULTI_AGENT.PUBLIC.DOC_TEXT_TABLE
)
SELECT
    relative_path,
    c.value AS chunk,
    (relative_path || ' ' || chunk) AS SEARCH_COL
FROM text_chunks,
LATERAL FLATTEN(INPUT => chunks) c;

In [ ]:
chunk_df = session.table('TR_MULTI_AGENT.PUBLIC.DOC_CHUNKS_TABLE')
chunk_df.show(3)

In [ ]:
CREATE OR REPLACE CORTEX SEARCH SERVICE TR_MULTI_AGENT.PUBLIC.PRODUCT_INFO_SEARCH
    ON SEARCH_COL
    WAREHOUSE = DEFAULT_XS
    TARGET_LAG = '365 days'
    AS SELECT 
        RELATIVE_PATH,
        CHUNK,
        SEARCH_COL
FROM TR_MULTI_AGENT.PUBLIC.DOC_CHUNKS_TABLE

In [ ]:
from typing import List
from snowflake.core import Root
product = "Cortex Search"
prompt = f"Retrieve relevant documentation for {product}"

root = Root(session)
cortex_search_service = (
    root
    .databases["TR_MULTI_AGENT"]
    .schemas["PUBLIC"]
    .cortex_search_services["PRODUCT_INFO_SEARCH"]
)
resp = cortex_search_service.search(
    query=prompt,
    columns=["SEARCH_COL"],
    limit=5)
    
text_chunks = [row["SEARCH_COL"] for row in resp.results] if resp.results else []

LLM_prompt = f"Can you summarize product benefits, value props, and a customer reference for product - {product} \
                using the following contextual information - {text_chunks}"
llm_response = complete("snowflake-llama-3.3-70b", LLM_prompt)

llm_response

In [ ]:
client = "Wealthsimple"

# resp = CA.process_message(f"What products are being used by our client {client}")
# products_used = resp.iloc[:,0].to_list()
products_used = session.sql(f"SELECT PRODUCTS_USED FROM CUSTOMER_DATA WHERE CUSTOMER= '{client}'").collect()[0][0]

suggested_product = session.sql(f"SELECT PROPENSITY_MODEL_SUGGESTED_PRODUCT FROM CUSTOMER_DATA WHERE CUSTOMER= '{client}'").collect()[0][0]

prompt = f"Retrieve relevant documentation for {product}"

root = Root(session)
cortex_search_service = (
    root
    .databases["TR_MULTI_AGENT"]
    .schemas["PUBLIC"]
    .cortex_search_services["PRODUCT_INFO_SEARCH"]
)
resp = cortex_search_service.search(
    query=prompt,
    columns=["SEARCH_COL"],
    limit=5)

text_chunks = [row["SEARCH_COL"] for row in resp.results] if resp.results else []

LLM_prompt = f"Can you summarize product benefits, value props, and a customer reference for product - {suggested_product} \
                using the following contextual information - {text_chunks}"
llm_response = complete("snowflake-llama-3.3-70b", LLM_prompt)


full_response = f'''The following products are currently being used by {client} - {products_used} \n\n
Our propensity model has suggested the following product - SUGGESTED PRODUCT: **{suggested_product}** \n\n
{llm_response}'''


# full_response

st.write(f"# Prep Report for Client **{client}**" + \
                       "\n\n ## Products Used \n\n" + full_response)

In [ ]:
from snowflake.cortex import complete 

class Multi_Agent_App:

    # def __init__(self):

    def agent_1_cortex_analyst(self, client: str) -> str:
        """
        takes client and inserts into predefined prompt and passes to cortex analyst to return list of products client uses today
        """
        # resp = CA.process_message(f"What products are being used by our client {client}")
        # products_used = resp.iloc[:,0].to_list()

        # Use sql to get list of currently used products and propensity-model-suggested product
        products_used = session.sql(f"SELECT PRODUCTS_USED FROM CUSTOMER_DATA WHERE CUSTOMER= '{client}'").collect()[0][0]
        suggested_product = session.sql(f"SELECT PROPENSITY_MODEL_SUGGESTED_PRODUCT FROM CUSTOMER_DATA WHERE CUSTOMER= '{client}'").collect()[0][0]

        #Retrieve relevant docuemntation for this product
        prompt = f"Retrieve relevant documentation for {product}"
        root = Root(session)
        cortex_search_service = (
            root
            .databases["TR_MULTI_AGENT"]
            .schemas["PUBLIC"]
            .cortex_search_services["PRODUCT_INFO_SEARCH"]
        )
        resp = cortex_search_service.search(
            query=prompt,
            columns=["SEARCH_COL"],
            limit=5)
    
        text_chunks = [row["SEARCH_COL"] for row in resp.results] if resp.results else []

        #Pass document chunks to LLM to get a product value prop for suggested product
        LLM_prompt = f"Can you summarize product benefits, value props, and a customer reference for product - {suggested_product} \
                        using the following contextual information - {text_chunks}"
        llm_response = complete("snowflake-llama-3.3-70b", LLM_prompt)
        

        full_response = f'''The following products are used by {client} - {products_used} \n\n
        
        \n Our propensity model has suggested the following product - SUGGESTED PRODUCT: **{suggested_product}** \n\n
        
        \n {llm_response}'''

        return full_response

    def agent_2_client_web_search(self, client: str) -> list:
        """ 
        calls api to google client name and return top three news articles. 
        Potentially use bs4 or similar library to parse web results
        Return list of web page text (or potentially pass to LLM to generate summary)
        """

        # api_key = os.environ.get("GOOGLE_API_KEY")
        # cse_id = os.environ.get("GOOGLE_CSE_ID")

        # web_query = f"top news for {client}"
        # url = "https://www.googleapis.com/customsearch/v1"
        # params = {
        #     "key": api_key,
        #     "cx": cse_id,
        #     "q": web_query,
        #     "num": 3
        # }

        # top_3_results = requests.get(url, params=params)

        # news_summary_list = []

        # for result in top_3_result:
        #     news_summaries_list.append(complete('snowflake-llama-3.3-70b', f'summarize the following web page {result}'))

        # return news_summaries_list
        news_summaries_list = []
        news_summaries_list.append(complete('llama3.2-1b', f'generate a brief news summary for client {client}. \
        It can be fictious but should be within the realm of what that client actually does. \
        If it is not a known client you can just make something up that seems reasonable for a tech company'))
        return news_summaries_list


    def agent_3_market_web_search(self, client: str) -> list:
        """ 
        uses an llm to determine client industry, then ues api to google client industry and return top three articles. 
        Potentially use bs4 or similar library to parse web results
        Return list of web page text (or potentially pass to LLM to generate summary)
        """
        # api_key = os.environ.get("GOOGLE_API_KEY")
        # cse_id = os.environ.get("GOOGLE_CSE_ID")

        # client_market = complete('snowflake-llama-3.3-70b', f'what is the most applicable market for the company {client}')

        # web_query = f"market news for {client_market}"
        # url = "https://www.googleapis.com/customsearch/v1"
        # params = {
        #     "key": api_key,
        #     "cx": cse_id,
        #     "q": web_query,
        #     "num": 3
        # }

        # top_3_results = requests.get(url, params=params)

        # market_news_summary_list = []

        # for result in top_3_result:
        #     market_news_summary_list.append(complete('snowflake-llama-3.3-70b', f'summarize the following web page {result}'))
        market_news_summaries_list = []
        market_news_summaries_list.append(complete('llama3.2-1b', f'generate a brief market news summary for the market that client {client} operates in. \
        It can be fictious but should be within the realm of how that market actually operates. \
        If it is not a known client/market you can just assume the client operates in the information technology market'))
        return market_news_summaries_list

    def agent_4_prep_pitch(self, client: str, product_info: Optional[str], client_news: Optional[list], market_news: Optional[list]) -> str:
        """ 
        passes products, client news, and market news to an LLM to come up with a rough prep pitch

        """
        

        if product_info is None:
            product_info = self.agent_1_cortex_analyst(client)
        if client_news is None:
            client_news = self.agent_2_client_web_search(client)
        if market_news is None:
            market_news = self.agent_3_market_web_search(client)
        prep_pitch = complete('snowflake-llama-3.3-70b', 
                              f"""
                              prepare a product pitch for ONLY FOR THE SUGGESTED PRODUCT based on the following product info and the client and market news 
                              product_info: {product_info}
                              client_news: {client_news}
                              market_news: {market_news}
                              """)
        return prep_pitch
    
    def agent_5_disco_questions(self, client: str, product_info: Optional[str], client_news: Optional[list], market_news: Optional[list]) -> str:
        """ 
        passes products, client news, and market news to an LLM to come up with a list of appropriate discovery questions
        """
        
        if product_info is None:
            product_info = self.agent_1_cortex_analyst(client)
        if client_news is None:
            client_news = self.agent_2_client_web_search(client)
        if market_news is None:
            market_news = self.agent_3_market_web_search(client)
        disco_questions = complete('snowflake-llama-3.3-70b', 
                              f"""
                              prepare a set of discovery questions for a client meeting for the SUGGESTED PRODUCT and the client and market news 
                              product_info: {product_info}
                              client_news: {client_news}
                              market_news: {market_news}
                              """)
        return disco_questions


    def orchestrate_all_agents(self, client: str, output_file_name: str):
        """ 
        primary function that ties all agents together and writes out a final markdown file with all prep materials
        """
        print("Starting Sales Prep Agent Pipeline...")
        
        product_info = self.agent_1_cortex_analyst(client)
        print("Collected Product Info")
        
        client_news = self.agent_2_client_web_search(client)
        print("Collected Client News")

        market_news = self.agent_3_market_web_search(client)
        print("Collected Market News")

        pitch = self.agent_4_prep_pitch(client, product_info, client_news, market_news)
        print("Generated Pitch")
        
        disco_questions = self.agent_5_disco_questions(client, product_info, client_news, market_news)
        print("Generated Disco Questions")
        
        markdown_doc = f"# Prep Report for Client **{client}**" + \
                       "\n\n ## Product Info \n\n" + product_info + \
                       "\n\n ## Client News \n\n " + ' '.join(client_news)+ \
                       "\n\n ## Market News \n\n " + ' '.join(market_news) + \
                       "\n\n ## Proposed Pitch \n\n " + pitch + \
                       "\n\n ## Discovery Questions \n\n  " + disco_questions + \
                       "\n\n # END OF DOCUMENT - good luck with your meeting :)"

        with open(output_file_name, "wb") as f:
            f.write(markdown_doc.encode("utf-8"))

        print(f"Client prep file written to {output_file_name}!")

        return markdown_doc
        # , products, client_news, market_news, pitch, disco_questions

In [ ]:
test_mac = Multi_Agent_App()
test_md = test_mac.orchestrate_all_agents('Wealthsimple', 'test.md')

In [ ]:
st.write(test_md)

In [ ]:
download_button = st.download_button(f'Download Client Prep Report', open('test.md', 'r'), 
                                     file_name='test.md', use_container_width=True)
if download_button:
#    st.session_state.download_clicked = True  # Set flag in session state
#    if st.session_state.download_clicked:
    st.success("✅ File Downloaded!")

# Archive


In [ ]:
create or replace network rule CLIENT_RESEARCH_NR
  TYPE = 'HOST_PORT'
  MODE= 'EGRESS'
  VALUE_LIST = ('www.businessinsider.com', 'www.techcrunch.com', 'www.google.com', 'www.googleapis.com', 'docs.snowflake.com');

CREATE or replace EXTERNAL ACCESS INTEGRATION CLIENT_RESEARCH_EAI
  ALLOWED_NETWORK_RULES = (CLIENT_RESEARCH_NR)
  ENABLED = true;

-- Make sure you enable this external access integration for the notebook!

In [ ]:
import requests
from bs4 import BeautifulSoup

def google_search(query, num_results=10):
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.0.0 Safari/537.36"
    }
    
    # Format query for Google Search URL
    search_url = f"https://www.google.com/search?q={query.replace(' ', '+')}"
    
    # Fetch the page
    response = requests.get(search_url, headers=headers)
    
    if response.status_code != 200:
        print(f"Failed to fetch Google search results: {response.status_code}")
        return []
    
    soup = BeautifulSoup(response.text, "html.parser")
    
    # Extract search result links
    links = []
    
    for g in soup.find_all("a", href=True):
        href = g["href"]
        print(href)
        if href.startswith("/url?q="):  # Google uses "/url?q=" before the actual link
            link = href.split("&")[0].replace("/url?q=", "")  # Clean up URL
            links.append(link)
    
    return links[:num_results]

# Example usage
results = google_search("Snowflake Inc")
print(results)

In [ ]:
import requests
from bs4 import BeautifulSoup
r = requests.get("https://www.businessinsider.com/answers#snowflake")
# r = requests.get("https://www.techcrunch.com/?s=snowflake")
# r = requests.get("https://www.google.com/search?q=snowflake&oq=snowflake")

soup = BeautifulSoup(r.text, 'html.parser')

# Find all <a> tags with href attributes
links = soup.find_all('a', href=True)

# Extract and print the links
for link in links:
    url = link['href']
    text = link.get_text(strip=True)
    if 'snowflake' in text.lower():
        print(f"URL: {url}")
        print(f"Text: {text}")
        print("---")

In [ ]:
from googlesearch import search

def get_top_business_insider_links(search_term):
    """
    Performs a Google search for a term with site:businessinsider.com and returns the top 3 links.

    Args:
        search_term (str): The term to search for.

    Returns:
        list: A list of the top 3 Business Insider links, or an empty list if none found.
    """
    try:
        query = f"{search_term} site:businessinsider.com"
        search_results = list(search(query, num_results=3, stop=3, pause=2)) #pause added

        business_insider_links = []
        for url in search_results:
            if "businessinsider.com" in url:
                business_insider_links.append(url)

        return business_insider_links

    except Exception as e:
        print(f"Error during search: {e}")
        return []

# Example usage:
search_term = "Snowflake"
top_links = get_top_business_insider_links(search_term)

if top_links:
    for i, link in enumerate(top_links):
        print(f"Link {i+1}: {link}")
else:
    print("No Business Insider links found.")

search_term = "Tesla"
top_links = get_top_business_insider_links(search_term)

if top_links:
    for i, link in enumerate(top_links):
        print(f"Link {i+1}: {link}")
else:
    print("No Business Insider links found.")

In [ ]:
# from googlesearch import search
import requests
from bs4 import BeautifulSoup

def get_top_article_text(company_name):
    """
    Performs a Google search for a company name and returns the raw text of the
    top three articles.

    Args:
        company_name (str): The name of the company to search for.

    Returns:
        list: A list of strings, where each string is the raw text of an article,
              or an empty list if an error occurs.
    """
    try:

        article_texts = []
        for url in search_results:
            try:
                response = requests.get(url, timeout=10) #added timeout
                response.raise_for_status() # Raise HTTPError for bad responses (4xx or 5xx)
                soup = BeautifulSoup(response.content, 'html.parser')
                text = soup.get_text(separator=' ', strip=True) #added separator and strip
                article_texts.append(text)
            except requests.exceptions.RequestException as e:
                print(f"Error fetching URL {url}: {e}")
            except Exception as e:
                print(f"Error processing URL {url}: {e}")

        return article_texts

    except Exception as e:
        print(f"Error during search: {e}")
        return []

# Example usage:
company = "Snowflake"
article_texts = get_top_article_text(company)

if article_texts:
    for i, text in enumerate(article_texts):
        print(f"Article {i+1}:\n{text}\n{'-'*40}\n")
else:
    print("Could not retrieve article text.")






web_query = f"top news for Snowflake"
url = "https://www.googleapis.com/customsearch/v1"
params = {
    "key": '',
    "cx": '',
    "q": web_query,
    "num": 3
}

top_3_results = requests.get(url, params=params)
top_3_results